# Exercise 2.2: Helper functions
prepared by M.Hauser

In this exercise we will reproduce this famous plot from the IPCC report summary for policymakers (IPCC, 2013, Figure 10). It shows CO$_2$ emmissons versus annual global mean temperarture from CMIP 5 models (Taylor et al., 2012). The plot illustrates that the global mean temperature scales quite linearly with the cummulative Co$_2$ emmissions and the emmission scenario has a small  impact on this scaling. For simlicity we leave away the simulations with 1 % CO$_2$ increase per year.

We will use the basic matplotlib plotting functions, but will learn how to embed them in a function that can be used for each dataset.

<img src="../figures/FigSPM-10.jpg"  width="500">

The data (global mean temperature from all CMIP5 models and prescribed CO$_2$ concentrations) was prepared in another [notebook](./../data/prepare_CMIP5_tas_time_series.ipynb).

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import xarray as xr

%matplotlib inline

In [ ]:
# load data 
rcp26 = xr.open_dataset('../data/cmip5_tas_rcp26_ts.nc')
rcp45 = xr.open_dataset('../data/cmip5_tas_rcp45_ts.nc')
rcp60 = xr.open_dataset('../data/cmip5_tas_rcp60_ts.nc')
rcp85 = xr.open_dataset('../data/cmip5_tas_rcp85_ts.nc')

rcp85

### Exercise

In the original plot they use cummulative CO$_2$ emmisions on the x-axis. We will use CO$_2$ concentration here.

* Get `co2` and the surface air temerature anomaly (`tas_anom`) from `rcp85`.
* What's the shape of the `tas_anom`?
* Calculate the multimodel mean. As we use xarray, you can do that by calling `rcp85.tas_anom.mean('ens')`.
* Make a plot of `co2` and the multi model tas mean.


In [ ]:
f, ax = plt.subplots()

# co2 =
# tasmean = 

# ax.plot(...)

### Solution

In [ ]:
f, ax = plt.subplots()

co2 = rcp85.co2
tasmean = rcp85.tas_anom.mean('ens')

ax.plot(co2, tasmean)

### Calculating decadal averages

We can use `resample` to calculate 10-year averages.

In [ ]:
rcp26_decade = rcp26.resample(time='10A').mean()
rcp26_decade['year'] = rcp26_decade['time.year']

rcp45_decade = rcp45.resample(time='10A').mean()
rcp45_decade['year'] = rcp45_decade['time.year']

rcp60_decade = rcp60.resample(time='10A').mean()
rcp60_decade['year'] = rcp60_decade['time.year']

rcp85_decade = rcp85.resample(time='10A').mean()
rcp85_decade['year'] = rcp85_decade['time.year']

### Exercise

* Add the 10-year mean data:
  * Read & plot `co2` and the surface air temerature anomaly (`tas_anom`) from `rcp85_decade`.
  * Add markers to each datapoint of `tasmean_decade`.
* Why does the curve with annual means go further than the decadal means?


In [ ]:
f, ax = plt.subplots()

co2 = rcp85.co2
tasmean = rcp85.tas_anom.mean('ens')

ax.plot(co2, tasmean)

# co2_decade = ...
# tasmean_decade = ...

# ax.plot(...)

### Solution

In [ ]:
f, ax = plt.subplots()


co2 = rcp85.co2
tasmean = rcp85.tas_anom.mean('ens')

ax.plot(co2, tasmean)


co2_decade = rcp85_decade.co2
tasmean_decade = rcp85_decade.tas_anom.mean('ens')

ax.plot(co2_decade, tasmean_decade, marker='.')

# the last point of 'decadal' corresponds to the mean 2091 to 2100 & there is a trend.

### Exercise

With `xarray` you can select a time-range using `hist = rcp85_decade.sel(time=slice(None, '2010')`. You can then get `co2` and `tas_anom.mean('ens')` from `hist`.

* Select `'2010'` to `'2100'` from `rcp85_decade`.

* Plot `*_hist` (1970 to 2010) in black. 
* Plot `*_fut` (2010 to 2100) in red line (`color='#e31a1c'`).
* Add markers (`'o'`) and make sure the black marker gets plotted above the red marker.

In [ ]:
f, ax = plt.subplots()

# get historical data
hist = rcp85_decade.sel(time=slice(None, '2010'))
co2_hist = hist.co2
tasmean_hist = hist.tas_anom.mean('ens')

# get projections
# fut = 
# co2_fut = 
# tasmean_fut = 
# ...

# ax.plot(...)
# ax.plot(...)

### Solution

In [ ]:
f, ax = plt.subplots()

# get historical data
hist = rcp85_decade.sel(time=slice(None, '2010'))
co2_hist = hist.co2
tasmean_hist = hist.tas_anom.mean('ens')

fut = rcp85_decade.sel(time=slice('2010', None))
co2_fut = fut.co2
tasmean_fut = fut.tas_anom.mean('ens')             
              

ax.plot(co2_fut, tasmean_fut, color='#e31a1c', marker='o')                     
ax.plot(co2_hist, tasmean_hist, color='0.1', marker='o')

### Plot function

We just wrote the same code twice... (Except for the selected years). This is pretty annoying... *And*, we would now need to add this code for every RCP scenario we want to plot. To our rescue, we can define a small function to do the selection and plotting for us:

``` python

def add_line(data, color, ax, time=slice('2010', None)):
    """select time period and plot mean tas"""

    # select time period
    data = data.sel(time=time)

    # get data
    year = data.year
    tasmean = data.tas_anom.mean('ens')
    
    # add plot
    ax.plot(year, tasmean, color=color)

```

#### Comments

* I assume you know that
  * python functions are constructed as `def name(argument, keyord_argument='default'):`
  * they need four spaces indentation. 
* The function expects three arguments, `data`, `color`, and `ax`.
* `time` has a default value, because most of the calls will be for this time period.


### Exercise
* Try it out
* Add the other three RCPs (only the time period 2010 to 2100, the historical data is the same for all of them.)
* Use the following colors
  * rcp60: '#ff7f00'
  * rcp45: '#1f78b4'
  * rcp26: '#a6cee3'
* Note: I think it does not make sense that RCP 4.5 has the brighter color than RCP 2.6, so I swapped them.
* Matlab users: appreciate how easy it is to define default values in python.

In [ ]:
def add_line(data, color, ax, time=slice('2010', None)):
    """select time period and plot mean tas"""

    data = data.sel(time=time)

    co2 = data.co2
    tasmean = data.tas_anom.mean('ens')
    
    ax.plot(co2, tasmean, color=color)
    
# ==============================================================
    
f, ax = plt.subplots()

# add projections
add_line(rcp85_decade, '#e31a1c', ax)

# add historical period (time=slice(....))


### Solution

In [ ]:
       
def add_line(data, color, ax, time=slice('2010', None)):
    """select time period and plot mean tas"""

    data = data.sel(time=time)

    co2 = data.co2
    tasmean = data.tas_anom.mean('ens')
    
    ax.plot(co2, tasmean, color=color)
    
# ==============================================================
    
f, ax = plt.subplots()


# add projections
add_line(rcp85_decade, '#e31a1c', ax)
add_line(rcp60_decade, '#ff7f00', ax)
add_line(rcp45_decade, '#1f78b4', ax)
add_line(rcp26_decade, '#a6cee3', ax)

# add historical period
add_line(rcp85_decade, '0.1', ax, time=slice(None, '2010'))

### Refining the function

This is already better, but we are still missing some things:
 * We need to add the marker.
 * What if we want to change other properties, such as the linewidth? Do we have to add a keyword argument for all of them?
 * What if we later require the return value of `ax.plot`?
 * Also, I would like to be able to do `add_line(rcp85_decade, 'm')`, i.e. I don't want to have to specify the `ax`.
 
#### Not explicitely requiring axes

When writing such a function it is very common (citation needed) to set `def fcn(..., ax=None, ...)`, as not to require this keyword. In the function we then check `if ax is None` and set it to `ax = plt.gca()` if needed. `gca` stands for get current axes - if an axes instance exists it gets it, if no axes exist it creates a new figure and axes.

#### Return value of `ax.plot`

Just do `return ax.plot(...)`. 


#### **kwargs

To tackle the second point we can make use of `**kwargs` (= keyword arguments, named arguments) to let your functions take an arbitrary number of keyword arguments (`**` unpacks dictionaries in python). We can then pass the `**kwargs` on to the plotting function. See below.

#### Marker

For the marker we could just add `marker='o'` to the arguments of the function. However your function definition can get very long. So we can make use of the fact that `kwargs` is a dict - we can set a value if it is not yet in the dict, using
`kwargs['key'] = kwargs.pop('key', 'default')` - see an example below.


#### plot function skeleton

Therefore we get the follwing

``` python

def pltfunc(data, ax=None, **kwargs):
    """select time period and plot mean tas"""

    if ax is None:
        ax = plt.gca()

    # DO STUFF

    return ax.plot(year, tasmean, **kwargs)

```

#### Note
Of course deciding between requiring a value (`def fcn(color):`),  setting a default value (`def fcn(color='r'):`), using kwargs and pop to set a default (`kwargs['color'] = kwargs.pop('color', 'r')`), or not setting the value at all (and let the user set it via `**kwargs`) is mostly a matter of taste.

### Showcase **kwargs

In [ ]:
def func(**kwargs):
    print(kwargs)
    print(type(kwargs))
    
    
# kwargs is an empty dict
print('func():')
func()

print()
print("func(a='a', color='red', t=5)")
# kwargs is a dict with all specified, named arguments
func(a='a', color='red', t=5)

### Showcase dict.pop

In [ ]:
# create an empty dict
d = dict()

print('Uses the default value:')
print(d.pop('color', 'r'))

d['color'] = 'b'

print('The dict now contains an entry:')
print(d)

print('Uses the value in the dict:')
print(d.pop('color', 'r'))

print("The entry is gone (that's what pop does):")
print(d)
print('If the entry should stay in the dict, use `get`.')

In [ ]:
# again
d = dict()

print('d is empty:')
print(d)

d['color'] = d.pop('color', 'b')

print("Uses 'b' as the dict is empty & fills the dict.")
print(d)

d['color'] = d.pop('color', 'r')

print("Uses 'b' and not 'r' as the key already exists.")
print(d)

### Refined function:

Implementing all the things we have learned, we end up with the following function:

``` python

def add_line(data, color, ax=None, time=slice('2010', None), **kwargs):
    """select time period and plot mean tas"""

    kwargs['marker'] = kwargs.pop('marker', 'o')
    
    if ax is None:
        ax = plt.gca()

    # get data    
    data = data.sel(time=time)

    co2 = data.co2
    tasmean = data.tas_anom.mean('ens')

    return ax.plot(co2, tasmean, color=color, **kwargs)

```


### Exercise

* Try it out

In [ ]:
def add_line(data, color, ax=None, time=slice('2010', None), **kwargs):
    """select time period and plot mean tas"""

    kwargs['marker'] = kwargs.pop('marker', 'o')
    
    if ax is None:
        ax = plt.gca()

    # get data    
    data = data.sel(time=time)

    co2 = data.co2
    tasmean = data.tas_anom.mean('ens')

    return ax.plot(co2, tasmean, color=color, **kwargs)

# ==============================================================
        
f, ax = plt.subplots()

# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax)
add_line(rcp60_decade, color='#ff7f00', ax=ax)
add_line(rcp45_decade, color='#1f78b4', ax=ax)
add_line(rcp26_decade, color='#a6cee3', ax=ax)

# add historical period
add_line(rcp85_decade, color='0.1', time=slice(None, '2010'), ax=ax)

### Defining the function in a external file

When you want to use your helper function in multiple notebooks, you need to add it to a sperate \*.py file. Have a look at the `utils.py` file in the current folder (`pyvis/Part2`). It contains the `add_line` function.


In [ ]:
import utils

In [ ]:
utils.add_line?

In [ ]:
utils.add_line??

### Exercise
 * Change the following code such that you use `add_line` from `utils`.
 * It should print "Using \`add_line\` from \`utils\`." twice.

In [ ]:
f, ax = plt.subplots()

# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax)

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax)

### Solution

In [ ]:
f, ax = plt.subplots()

# add projections
utils.add_line(rcp85_decade, color='#e31a1c', ax=ax)

# add historical period
utils.add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax)

### Adapt the function

The function is not yet entirely correct.

 * Change `utils.py`:
   * I forgot the line `kwargs['marker'] = kwargs.pop('marker', 'o')`. Add it.
   * Remove the annoying print statement.
   * Don't forget to save your changes
 * Try if your changes took effect:

In [ ]:
f, ax = plt.subplots()

# add projections
utils.add_line(rcp85_decade, color='#e31a1c', ax=ax)

# add historical period
utils.add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax)

### Nothing changed

Once the source code has been imorted, it does not change even when you change the file it came from. Now you can
 * restart the notebook (Kernel -> Restart), and rerun the whole notebook, this is the cleanest solution, but it may take a while until you reloaded the data.
 * use `reload` to explicitely tell python that your source code has changed.

In [ ]:
# import the reload capability (needed in python3)

from importlib import reload

# reload the script with your notifications
reload(utils)

In [ ]:
f, ax = plt.subplots()

# add projections
utils.add_line(rcp85_decade, color='#e31a1c', ax=ax)

# add historical period
utils.add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax)

### Adding the uncertainty bands (range)

Adding the uncertainty bands is quite tricky, because the x-axis is in CO$_2$ and not in time (see [Exercise 1.1](ex1_1_first_plot_CO2_CMIP5.ipynb)). We will bin all tas_anom data (remember that tas_anom is made up from many models) and calculate the minimum and maximum. For this we can use the [groupby_bins function](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.groupby_bins.html) of `xarray`.

We will plot the range for each RCP individually. 

> Important: we do that with `rcpXX` and not with `rcpXX_decadal`, i.e. the annual data and not the decadal data.


In [ ]:
# create the bins

co2_bins = np.arange(280, 960, 20)
co2_bin_centers = (co2_bins[1:] + co2_bins[:-1]) / 2

print('bins:', co2_bins[:3].tolist() + ['...'])
print('bin centers:',  co2_bin_centers[:2].tolist() + ['...'])

print()
print('shape:')

print('bins:', co2_bins.shape)
print('bin centers:', co2_bin_centers.shape)

In [ ]:
# not repeating the definition of add_line,
# as we will not change it anymore

# ==============================================================

f, ax = plt.subplots()

# ==============================================================

# add range for rcp45

# calculate minimum and maximum in each co2_bin
mn = rcp45.groupby_bins('co2', co2_bins).min().tas_anom
mx = rcp45.groupby_bins('co2', co2_bins).max().tas_anom

ax.fill_between(co2_bin_centers, mn, mx, zorder=0, color='#ed9ca2')

# ==============================================================

# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax)
add_line(rcp60_decade, color='#ff7f00', ax=ax)
add_line(rcp45_decade, color='#1f78b4', ax=ax)
add_line(rcp26_decade, color='#a6cee3', ax=ax)

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax)

### Exercise

* Write a function `shade_range`.

In [ ]:
# not repeating the definition of add_line,
# as we will not change it anymore

def shade_range(data, color='#ed9ca2', ax=None, **kwargs):
    # your code here
    
#     mn = 
#     mx = 
#     ax.fill_between(
    
    pass

# ==============================================================

f, ax = plt.subplots()
slice(None, '2010') 

# add range
shade_range(rcp26, ax=ax)
shade_range(rcp45, ax=ax)
shade_range(rcp60, ax=ax)
shade_range(rcp85, ax=ax)


# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax)
add_line(rcp60_decade, color='#ff7f00', ax=ax)
add_line(rcp45_decade, color='#1f78b4', ax=ax)
add_line(rcp26_decade, color='#a6cee3', ax=ax)

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax)

### Solution

In [ ]:

def shade_range(data, color='#ed9ca2', ax=None, **kwargs):
    
    if ax is None:
        ax = plt.gca()

    mn = data.groupby_bins('co2', co2_bins).min().tas_anom
    mx = data.groupby_bins('co2', co2_bins).max().tas_anom
    
    return ax.fill_between(co2_bin_centers, mn.values, mx.values, color=color, zorder=0)

# ==============================================================

f, ax = plt.subplots()

# add uncertainty
shade_range(rcp26, ax=ax)
shade_range(rcp45, ax=ax)
shade_range(rcp60, ax=ax)
shade_range(rcp85, ax=ax)


# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax)
add_line(rcp60_decade, color='#ff7f00', ax=ax)
add_line(rcp45_decade, color='#1f78b4', ax=ax)
add_line(rcp26_decade, color='#a6cee3', ax=ax)

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax)

### Exercise

* add x & y labels
* add a title (e.g. `'Temperature - CO2 scaling'`)
* restrict the y-limit to -0.5..5
* add a legend (don't forget `label='XXX'`)

In [ ]:
f, ax = plt.subplots()

# add uncertainty
shade_range(rcp26, ax=ax)
shade_range(rcp45, ax=ax)
shade_range(rcp60, ax=ax)
shade_range(rcp85, ax=ax)


# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax)
add_line(rcp60_decade, color='#ff7f00', ax=ax)
add_line(rcp45_decade, color='#1f78b4', ax=ax)
add_line(rcp26_decade, color='#a6cee3', ax=ax)

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax)

# code here


### Solution

In [ ]:
f, ax = plt.subplots()

# add uncertainty
shade_range(rcp26, ax=ax)
shade_range(rcp45, ax=ax)
shade_range(rcp60, ax=ax)
shade_range(rcp85, ax=ax)


# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax, label='RCP 8.5')
add_line(rcp60_decade, color='#ff7f00', ax=ax, label='RCP 6.0')
add_line(rcp45_decade, color='#1f78b4', ax=ax, label='RCP 4.5')
add_line(rcp26_decade, color='#a6cee3', ax=ax, label='RCP 2.6')

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax, label='Historical')


ax.set_title("Temperature - CO2 scaling")
ax.set_ylabel('T anom (wrt 1861 to 1881) [K]')
ax.set_xlabel('CO$_2$ [ppm]')


ax.set_ylim(-0.5, 5)

ax.legend()

### Exercise

 * Add the year labels to the curves, see original figure (e.g. '1900', '2000', '2010', '2050', '2100').
 I prepared a function to read the temperature and co2 from the date, given the year.
 You can do this with differing complexity:
   * simple solution using `ax.text` (play with `ha` and `va` to avoid overlapping)
   * advanced solution with `ax.annotate` and adjusting the offset, depending on whether we are to the right or the left of the curve.
 

In [ ]:
f, ax = plt.subplots()

# add uncertainty
shade_range(rcp26, ax=ax)
shade_range(rcp45, ax=ax)
shade_range(rcp60, ax=ax)
shade_range(rcp85, ax=ax)


# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax, label='RCP 8.5')
add_line(rcp60_decade, color='#ff7f00', ax=ax, label='RCP 6.0')
add_line(rcp45_decade, color='#1f78b4', ax=ax, label='RCP 4.5')
add_line(rcp26_decade, color='#a6cee3', ax=ax, label='RCP 2.6')

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax, label='Historical')


ax.set_title("Temperature - CO2 scaling")
ax.set_ylabel('T anom (wrt 1861 to 1881) [K]')
ax.set_xlabel('CO$_2$ [ppm]')


ax.set_ylim(-0.5, 5)

ax.legend()

# ==============================================================

def add_year(ax, data, time, color, **kwargs):

   # convert time (2000) to a string ('2000')
    str_time = str(time)
    
    # get tas_anom and co2 at time
    tas_anom = data.tas_anom.sel(time=str(time)).mean('ens')
    co2 = data.co2.sel(time=str(time))
    
    kwargs['fontsize'] = kwargs.pop('fontsize', 8)
    
    # code here
    # ax.text(...)

# ==============================================================

# call of the function here
    
# rcps
add_year(ax, rcp85_decade, 2050, '#e31a1c', va='top')
add_year(ax, rcp85_decade, 2100, '#e31a1c', va='top')

add_year(ax, rcp60_decade, 2050, '#ff7f00', va='top')
add_year(ax, rcp60_decade, 2100, '#ff7f00', va='top')

add_year(ax, rcp45_decade, 2050, '#1f78b4', ha='right')
add_year(ax, rcp45_decade, 2100, '#1f78b4', ha='right')

add_year(ax, rcp26_decade, 2100, '#a6cee3', ha='right')

# observations
add_year(ax, rcp26_decade, 2010, '0.1', ha='right')
add_year(ax, rcp26_decade, 2000, '0.1', ha='right')
add_year(ax, rcp85_decade, 1950, '0.1', va='top')
add_year(ax, rcp85_decade, 1900, '0.1', va='top')

### Solution (ax.text, simple)

the colors should be tweaked a bit

In [ ]:
f, ax = plt.subplots()

# add uncertainty
shade_range(rcp26, ax=ax)
shade_range(rcp45, ax=ax)
shade_range(rcp60, ax=ax)
shade_range(rcp85, ax=ax)


# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax, label='RCP 8.5')
add_line(rcp60_decade, color='#ff7f00', ax=ax, label='RCP 6.0')
add_line(rcp45_decade, color='#1f78b4', ax=ax, label='RCP 4.5')
add_line(rcp26_decade, color='#a6cee3', ax=ax, label='RCP 2.6')

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax, label='Historical')


ax.set_title("Temperature - CO2 scaling")
ax.set_ylabel('T anom (wrt 1861 to 1881) [K]')
ax.set_xlabel('CO$_2$ [ppm]')


ax.set_ylim(-0.5, 5)

ax.legend()


time = 2100


def add_year(ax, data, time, color, **kwargs):

   # convert time (2000) to a string ('2000')
    str_time = str(time)
    
    # get tas_anom and co2 at time
    tas_anom = data.tas_anom.sel(time=str(time)).mean('ens')
    co2 = data.co2.sel(time=str(time))
    
    kwargs['fontsize'] = kwargs.pop('fontsize', 8)
    
    ax.text(co2, tas_anom, str_time, color=color, **kwargs)
    

# rcps
add_year(ax, rcp85_decade, 2050, '#e31a1c', va='top')
add_year(ax, rcp85_decade, 2100, '#e31a1c', va='top')

add_year(ax, rcp60_decade, 2050, '#ff7f00', va='top')
add_year(ax, rcp60_decade, 2100, '#ff7f00', va='top')

add_year(ax, rcp45_decade, 2050, '#1f78b4', ha='right')
add_year(ax, rcp45_decade, 2100, '#1f78b4', ha='right')

add_year(ax, rcp26_decade, 2100, '#a6cee3', ha='right')

# observations
add_year(ax, rcp26_decade, 2010, '0.1', ha='right')
add_year(ax, rcp26_decade, 2000, '0.1', ha='right')
add_year(ax, rcp85_decade, 1950, '0.1', va='top')
add_year(ax, rcp85_decade, 1900, '0.1', va='top')

### Solution (ax.annotate, fancy)

the colors should be tweaked a bit

In [ ]:
f, ax = plt.subplots()

# add uncertainty
shade_range(rcp26, ax=ax)
shade_range(rcp45, ax=ax)
shade_range(rcp60, ax=ax)
shade_range(rcp85, ax=ax)


# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax, label='RCP 8.5')
add_line(rcp60_decade, color='#ff7f00', ax=ax, label='RCP 6.0')
add_line(rcp45_decade, color='#1f78b4', ax=ax, label='RCP 4.5')
add_line(rcp26_decade, color='#a6cee3', ax=ax, label='RCP 2.6')

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax, label='Historical')


ax.set_title("Temperature - CO2 scaling")
ax.set_ylabel('T anom (wrt 1861 to 1881) [K]')
ax.set_xlabel('CO$_2$ [ppm]')


ax.set_ylim(-0.5, 5)

ax.legend()


time = 2100


def add_year(ax, data, time, color, right=True):

    # convert time (2000) to a string ('2000')
    str_time = str(time)
    
    # get tas_anom and co2 at time
    tas_anom = data.tas_anom.sel(time=str(time)).mean('ens')
    co2 = data.co2.sel(time=str(time))

    x_offset = 3 if right else -3

    ha = 'left' if right else 'right'
    va = 'top' if right else 'bottom'
    
    
    ax.annotate(str_time,
                xy=(co2, tas_anom),
                xytext=(x_offset, -2), textcoords='offset points',
                va=va,
                ha=ha,
                fontsize=8,
                color=color)
    
    

add_year(ax, rcp85_decade, 2050, '#e31a1c')
add_year(ax, rcp85_decade, 2100, '#e31a1c')

add_year(ax, rcp60_decade, 2050, '#ff7f00')
add_year(ax, rcp60_decade, 2100, '#ff7f00')

add_year(ax, rcp45_decade, 2050, '#1f78b4', right=False)
add_year(ax, rcp45_decade, 2100, '#1f78b4')

add_year(ax, rcp26_decade, 2100, '#a6cee3', right=False)

add_year(ax, rcp26_decade, 2010, '0.1', right=False)
add_year(ax, rcp26_decade, 2000, '0.1', right=True)
add_year(ax, rcp26_decade, 1950, '0.1', right=False)
add_year(ax, rcp26_decade, 1900, '0.1', right=True)

## Bonus: Color gradient

Ok, I know you want it - the color gradient. Unfortunately there is no possibility to add it automatically, so it's not easy to do - I follow a recipie from a [blog](http://pradhanphy.blogspot.ch/2014/06/filling-between-curves-with-color.html).

Instead of using `fill_between`, we will add a rgb-image with a color gradient. With the original color on the left (`'#ed9ca2'` = `(0.93, 0.61, 0.63)`) to white on the right (`'#ffffff'` = `(1, 1, 1)`). Then we clip the image to the extent of the range. 

In [ ]:
# we need to import Path an PathPatch

import matplotlib as mpl
from matplotlib.path import Path
from matplotlib.patches import PathPatch

First we need a function to encode the colors values. We could do it linearly, but let's choose a fancier function: it should be relatively constant at first and then show a strong increase to the right. We choose a function of the form

> $\alpha = \frac{\exp(\mathrm{co2} * \gamma)}{\exp(\max(\mathrm{co2}) * \gamma)}$

$\gamma$ is determinded by testing various values.

In [ ]:
co2_min = co2_bin_centers.min()
co2_max = co2_bin_centers.max()

co2 = np.linspace(co2_min, co2_max, 600)

gamma = 0.005

alpha = np.exp(co2 * gamma) / np.exp(co2_max * gamma)


plt.plot(co2, alpha)

plt.xlabel('CO2')
plt.ylabel('alpha channel');

Next we need to create an array with the colors.

In [ ]:
# convert the hexadecimal color to rgb
r, g, b, _ = mpl.colors.to_rgba('#ed9ca2')

# create an empty rgba array
color_gradient = np.ones([1, len(alpha), 3])

# assign the r, g, and b values
color_gradient[:, :, 0] = r + (1 - r) * alpha
color_gradient[:, :, 1] = g + (1 - g) * alpha
color_gradient[:, :, 2] = b + (1 - b) * alpha

print(color_gradient.shape)

color_gradient[0, :]

The `color_gradient` array has the dimensions `(y, x, color)`.

We use `ax.imshow` to add the constructed color_gradient. This function plots images, it assumes each rgb value belongs to a pixel.

In [ ]:
f, ax = plt.subplots()

ax.imshow(color_gradient, extent=[co2.min(), co2.max(), -2, 7], origin='lower') 
ax.set_aspect('auto')


# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax)

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax)

According to the recipie, we first need to create a `Path`, then from this path, a `Patch` and then we need to clip the image to this patch. Let's start by creating a path manually:

In [ ]:
f, ax = plt.subplots()

# manually create a path
p = [[400, 0], [900, 0], [900, 4], [400, 4]]
path = Path(p)

# create an invisible patch
patch = PathPatch(path, facecolor='none', edgecolor='none')

# we need to draw the patch, else 
ax.add_patch(patch)

ax.imshow(color_gradient, extent=[co2.min(), co2.max(), -2, 7], origin='lower', clip_on=True,
          clip_path=patch) 

ax.set_aspect('auto')

# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax)

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax)

Next we need to get the path of the range from `fill_between` to clip the picture to the range of the data. We also put everything into a function:

In [ ]:

def shade_range_gradient(data, color_gradient=color_gradient, ax=None, **kwargs):
    
    if ax is None:
        ax = plt.gca()

    mn = data.groupby_bins('co2', co2_bins).min().tas_anom
    mx = data.groupby_bins('co2', co2_bins).max().tas_anom
    
    # make it invisible by using color='none
    x = ax.fill_between(co2_bin_centers, mn.values, mx.values, color='none', zorder=0)
    
    # extract tha path from fill_between
    # we could also create the path manually, but this is easier
    path = x.get_paths()
    
    # create an invisible patch
    patch = PathPatch(path[0], facecolor='none', edgecolor='none')
    
    # we need to draw the patch, else the position is not correct
    ax.add_patch(patch)
    
    ax.imshow(color_gradient, extent=[co2.min(), co2.max(), -2, 7], origin='lower', clip_on=True,
              clip_path=patch)    
    
    ax.set_aspect('auto')
    
    return 

# ==============================================================

f, ax = plt.subplots()

shade_range_gradient(rcp85, ax=ax)
shade_range_gradient(rcp60, ax=ax)
shade_range_gradient(rcp45, ax=ax)
shade_range_gradient(rcp26, ax=ax)


# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax)
add_line(rcp60_decade, color='#ff7f00', ax=ax)
add_line(rcp45_decade, color='#1f78b4', ax=ax)
add_line(rcp26_decade, color='#a6cee3', ax=ax)

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax)

## Final Plot

Now we only need to add the legend, title, etc again:

In [ ]:
# shade range does not change

# ==============================================================

f, ax = plt.subplots()

shade_range_gradient(rcp85, ax=ax)
shade_range_gradient(rcp60, ax=ax)
shade_range_gradient(rcp45, ax=ax)
shade_range_gradient(rcp26, ax=ax)


# add projections
add_line(rcp85_decade, color='#e31a1c', ax=ax, label='RCP 8.5')
add_line(rcp60_decade, color='#ff7f00', ax=ax, label='RCP 6.0')
add_line(rcp45_decade, color='#1f78b4', ax=ax, label='RCP 4.5')
add_line(rcp26_decade, color='#a6cee3', ax=ax, label='RCP 2.6')

# add historical period
add_line(rcp85_decade, '0.1', time=slice(None, '2010'), ax=ax, label='Historical')


ax.set_ylim(-0.5, 5)


ax.set_title("Temperature - CO2 scaling")
ax.set_ylabel('T anom (wrt 1861 to 1881) [K]')
ax.set_xlabel('CO$_2$ [ppm]')


ax.set_ylim(-0.5, 5)

ax.legend()


# use add_year to be consistent for all
utils.add_year(ax, rcp85_decade, 2050, '#e31a1c')
utils.add_year(ax, rcp85_decade, 2100, '#e31a1c')

utils.add_year(ax, rcp60_decade, 2050, '#ff7f00')
utils.add_year(ax, rcp60_decade, 2100, '#ff7f00')

utils.add_year(ax, rcp45_decade, 2050, '#1f78b4', right=False)
utils.add_year(ax, rcp45_decade, 2100, '#1f78b4')

utils.add_year(ax, rcp26_decade, 2100, '#a6cee3', right=False)

utils.add_year(ax, rcp26_decade, 2010, '0.1', right=False)
utils.add_year(ax, rcp26_decade, 2000, '0.1', right=True)
utils.add_year(ax, rcp26_decade, 1950, '0.1', right=False)
utils.add_year(ax, rcp26_decade, 1900, '0.1', right=True)

### Bonus exercise (no solution)

 * Create a colorgradient that fades out towards the top right edge of the plot.
